In [33]:
# Temporal
from datetime import datetime, timedelta
import holidays

# Data retrieval
from pandas_datareader import wb
import requests

# Arthmetic operations and data manipulation libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Hyperparameter optimisation libraries
import optuna
import mlflow

In [34]:
# NOTE: review the links for guidance on connecting to a managed tracking server, such as the Databricks managed MLflow

mlflow.set_tracking_uri("http://localhost:8080")

# Generate training data

In [47]:
def get_openmeteo_weather_chunked(start_date, end_date, lat=-37.8136, lon=144.9631):
    """Fetch historical daily weather in 1-year chunks to avoid API errors."""
    all_weather = []
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    current_start = start
    while current_start <= end:
        current_end = min(
            current_start + pd.DateOffset(years=1) - pd.Timedelta(days=1), end
        )
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": current_start.strftime("%Y-%m-%d"),
            "end_date": current_end.strftime("%Y-%m-%d"),
            "daily": "temperature_2m_mean,precipitation_sum",
            "timezone": "Australia/Melbourne",
        }
        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()
        chunk_df = pd.DataFrame(
            {
                "date": pd.to_datetime(data["daily"]["time"]),
                "average_temperature": data["daily"]["temperature_2m_mean"],
                "rainfall": data["daily"]["precipitation_sum"],
            }
        )
        all_weather.append(chunk_df)
        current_start = current_end + pd.Timedelta(days=1)

    return pd.concat(all_weather, ignore_index=True)

In [ ]:
# # test the function
# weather_data = get_openmeteo_weather_chunked("2015-01-01", "2025-08-13")
# print(weather_data.head())

HTTPError: 400 Client Error: Bad Request for url: https://archive-api.open-meteo.com/v1/archive?latitude=-37.8136&longitude=144.9631&start_date=2025-01-01&end_date=2025-08-14&daily=temperature_2m_mean%2Cprecipitation_sum&timezone=Australia%2FMelbourne

Generate a dataset for predicting apple sales, including multiple influencial factors.

Args:
- base_demand (int, optional): Base demand for apples. Defaults to 1000.
- n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000
- competitor_price_effect (float, optional): Effect of competitor's price being lower on our sales. Defaults to -50.

Returns: pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

In [ ]:
def generate_apple_sales_data_with_promo_adjustment(
    base_demand: int = 1000,
    n_rows: int = 365 * 10,  # ~10 years
    competitor_price_effect: float = -50.0,
):
    np.random.seed(42)

    # ------------------------
    # Date range
    # ------------------------
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()  # oldest first
    df = pd.DataFrame({"date": dates})

    # Remove timestamps to match Open-Meteo format
    df["date"] = df["date"].dt.floor("D")
    df["month"] = df["date"].dt.month

    # ------------------------
    # Weather from Open-Meteo (chunked)
    # ------------------------

    # Max allowed date for Open-Meteo
    max_allowed_date = datetime.now() - timedelta(days=1)
    start_date = df["date"].min().strftime("%Y-%m-%d")
    end_date = min(df["date"].max(), max_allowed_date).strftime("%Y-%m-%d")
    weather_df = get_openmeteo_weather_chunked(start_date, end_date)
    df = df.merge(weather_df, on="date", how="left")

    # Fill missing temperatures and rainfall by monthly average, then overall mean
    for col in ["average_temperature", "rainfall"]:
        df[col] = df.groupby("month")[col].transform(lambda x: x.fillna(x.mean()))
        df[col] = df[col].fillna(df[col].mean())

    # ------------------------
    # Public holidays (Victoria, AU)
    # ------------------------
    au_holiday = holidays.Australia(years=set(d.year for d in dates), prov="VIC")
    df["holiday"] = [1 if date in au_holiday else 0 for date in df["date"]]

    # ------------------------
    # Base features
    # ------------------------
    df["weekend"] = [(date.weekday() >= 5) * 1 for date in df["date"]]
    df["price_per_kg"] = np.random.uniform(0.5, 3.0, n_rows)


In [ ]:

    # ------------------------
    # Real Inflation
    # ------------------------
    years = sorted(set(df["date"].dt.year))
    inflation_data = wb.download(
        indicator="FP.CPI.TOTL.ZG", country="AU", start=min(years), end=max(years)
    ).reset_index()
    inflation_data["FP.CPI.TOTL.ZG"] = pd.to_numeric(
        inflation_data["FP.CPI.TOTL.ZG"], errors="coerce"
    )
    inflation_data.rename(columns={"FP.CPI.TOTL.ZG": "inflation_rate"}, inplace=True)

    cumulative_factors = {}
    multiplier = 1.0
    start_year = min(years)
    for y in range(start_year, max(years) + 1):
        rate = inflation_data.loc[inflation_data["year"] == y, "inflation_rate"]
        if not rate.empty:
            multiplier *= 1 + rate.iloc[0] / 100
        cumulative_factors[y] = multiplier
    df["inflation_multiplier"] = df["date"].dt.year.map(cumulative_factors)

    # ------------------------
    # Seasonality & Promotions
    # ------------------------
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )
    df["price_per_kg"] -= df["harvest_effect"] * 0.5
    peak_months = [4, 10]
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # ------------------------
    # Base demand
    # ------------------------
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df["inflation_multiplier"]

    # Previous day demand
    df["previous_days_demand"] = df["demand"].shift(1).bfill()

    # Competitor pricing
    df["competitor_price_per_kg"] = np.random.uniform(0.5, 3, n_rows)
    df["competitor_price_effect"] = (
        df["competitor_price_per_kg"] < df["price_per_kg"]
    ) * competitor_price_effect

    # Stock availability with 3-day lag
    log_decay = -np.log(df["price_per_kg"].shift(3) + 1) + 2
    df["stock_available"] = np.clip(log_decay, 0.7, 1)

    # Marketing intensity
    df["marketing_intensity"] = np.random.uniform(0, 0.5, n_rows)
    high_stock_indices = df[df["stock_available"] > 0.95].index
    for idx in high_stock_indices:
        df.loc[idx : min(idx + 7, n_rows - 1), "marketing_intensity"] = (
            np.random.uniform(0.7, 1)
        )
    df["demand"] += df["competitor_price_effect"] + df["marketing_intensity"]

    # ------------------------
    # Drop temporary columns
    # ------------------------
    df.drop(
        columns=[
            "inflation_multiplier",
            "harvest_effect",
            "month",
            "competitor_price_effect",
            "stock_available",
        ],
        inplace=True,
    )

    return df

In [58]:
df = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=5000)
df

/var/folders/b6/h7hpf0z56490jrxnbg4c6wb00000gn/T/ipykernel_73866/3459251889.py:51: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  inflation_data = wb.download(


,date,average_temperature,rainfall,holiday,weekend,price_per_kg,promo,demand,previous_days_demand,competitor_price_per_kg,marketing_intensity
0,2011-12-07,19.000000,0.000000,0,0,1.436350,0,854.126631,853.789400,1.579969,0.337231
1,2011-12-08,22.000000,0.000000,0,0,2.876786,0,750.048635,853.789400,0.641788,0.147228
2,2011-12-09,23.400000,0.000000,0,0,2.329985,1,1053.206073,799.901407,1.970232,0.264371
3,2011-12-10,21.900000,8.700000,0,1,1.996646,0,1142.268425,1102.941703,2.390737,0.794402
4,2011-12-11,15.800000,10.100000,0,1,0.890047,1,1511.923787,1141.474023,1.533281,0.794402
...,...,...,...,...,...,...,...,...,...,...,...
4995,2025-08-10,8.900000,0.000000,0,1,2.743493,1,1363.409515,1249.021602,1.225628,0.884040
4996,2025-08-11,10.100000,0.000000,0,0,0.798453,1,1158.848014,1412.525475,1.638700,0.884040
4997,2025-08-12,10.391063,1.800483,0,0,1.319607,1,1125.519022,1157.963974,1.342604,0.806787
4998,2025-08-13,10.391063,1.800483,0,0,2.539363,0,761.405443,1124.712234,1.869866,0.806787


# Verifying Feature-Target Correlations

This step is important for:

1. Avoiding Data Leakage: Features should be completely uncorrelated with the target (correlation coefficient of 1.0). If such correlation exists, it indicates that the dataset may be "leaking" information about the target. Using such data for hyperparameter tuning can lead to misleading results, as it is easy for models to achieve perfect scores without truly learning the underlying patterns.

2. Ensuring Meaningful Relationships: Ideally, features should have some degree of correlation with the target. A correlation coefficient close to 0 for all features indicates a weak linear relationship. While this does not necessarily render the feature useless, it does introduce the following challenges:

- Predictive Power: The model may struggle to make accurate predictions.
- Overfitting Risk: Weak correlations increase the risk of the model fitting noise rather than true patterns, leading to overfitting.
- Complexity: Showing nonlinear relationships and interactions between features requires more sophisticated visualization and evaluation.

3. Audit and traceability: This correlated visualization can be logged alongside the main MLflow run to ensure traceability and provide a snapshot of the data characteristics at the time of model training, which is useful for auditing and reproducibility.